# 1. Introduction
Sleep is crucial for individuals' health and well being, it progresses in cycles that involve multiple sleep stages : wake, light sleep, deep sleep, rem sleep and the monitoring of these stages is useful for detecting sleep disorders.\
The Dreem headband offers a solution to perform a polysomnography -which consists of the recording of different physiological signals such as electroencephalogram, electrocardiogram etc.- at home.\
In this challenge, we suggest methods to predict sleep stages (labeled from 0 to 4) on windows of 30 seconds of raw data based on the corresponding recorded signals. The raw data provided by the three sensors on the headband includes 7 eegs channels in frontal and occipital position, 1 pulse oximeter infrared channel, and 3 accelerometers channels (x, y and z). We manage to achieve a $F_1$ score of 0.764.



# 2. Features extraction 

In order to construct a design matrix for our machine learning processing, we extract several statistical features from from our eeg signals denoted $x = (x_{1},...x_{n})$. The extracted features are the following :

**Mean** :

For each eeg signal we compute :
$$ \bar{x} = \sum_{i = 1}^{n} \frac{1}{n} x_{i}$$
**Standard deviation**:

$$ \sigma(x) = \sqrt{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2} $$

**Percentiles of the data**:

Without loss of generality, suppose that we sorted the points of the signal so that $x_{1}$ is the smallest value and $x_{n}$ is the greatest value. $x_i$ is the $q_i$ percentile of the signal where :
$$  q_i = 100\frac{i-0.5}{n}$$
The 5th, 25th, 50th (median), 75th and 95th are all computed for each signal.

**Number of zero crossing**:

we count the number of times at which the value of signal is zero.
$$ zeros(x) = \sum_{i = 1}^{n} \delta_{x_i, 0}$$
**Number of mean crossing** :

we count the number of times at which the value of signal is the mean of the signal.
$$ zeros(x) = \sum_{i = 1}^{n} \delta_{x_i, \bar{x}}$$

**Root Mean Square** :

$$ rms(x) = \sqrt{\frac{1}{n}\sum_{i=1}^{n}x_{i}^2}$$

**Absolute energy** :
$$ abs(x) = \sum_{i =1}^{n}x_{i}^2 $$
** Inter-quartile range** :

we substract the first quartile from the third quartile :
$$ IQR = q_75 - q_25 $$

**Median absolute deviation** :

$$ MAD(x) = Median(| x - Median(x)|)$$
**Shanon entropy** :

In general, we can define the entropy of a discrete random variables $X$ with values in ${x_{1},...x_{n}}$ as follows :
$$ H(X) = -c\sum_{i = 1}^{n} p(x_{i})ln(p(x_{i}))$$
$ p(x_{i}) $ is the probability of occurrence of $x_{i}$.
In practice we take $c = 1$ and $ p(x_{i}) $ as the empirical occurrence frequency of $x_{i}$.

**Spectral entropy** : 

Given a sampling frequency $f_s$, The spectral entropy is nothing but the  Shanon entropy of the power spectral density of the signal PSD (Normalized).
$$ H(x,f_s) = -\sum_{ f = 0}^{f_s/2} PSD(f)ln(PSD(f))  $$

**Petrosian fractal dimension** :

$$ PFD = \frac{Log(n)}{Log(n) + \frac{Log(n)}{Log(n)+ 0.4N_{\Delta}}} $$
with : 
$$ N_{\Delta} = \frac{1}{2} \sum_{i = 1}^{n-1} 1 -sign(x_{i}x_{i+1})$$

**Hjorth parameters** :

Hjorth parameters consists in three statistical properties in the time domain.

**Activity** :

The activity is nothing but the variance of the signal 
$$ activity(x) = var(x(t)) $$

**Morbidity**:

The morbidity is a way to capture the mean frequency in the power spectrum :
$$ morbidity(x) = \sqrt{\frac{var(\frac{dx}{dt})}{var(x)}}$$

**Complexity** :

The complexity is a way to capture the change in frequency and is defined by the mean of the the mobility.
$$ complexity(x) = \frac{mobility(\frac{dx}{dt})}{morbidity(x)}$$ 

# 3. Construction of the design matrix : Python Implementation

All the features are implemented in python. For that purpose, we define several functions that extract features for each signal. We regroup the straightforward standard statistcs in one function called *calculate_statistics(list_values)*, we group in another function called *calculate_crossings(list_values)* the zero and mean crossing. For the rest, each feature is defined as an independent function. 

In [ ]:
import numpy as np
from numba import jit
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
import pandas as pd
from collections import Counter
import scipy.stats
import numpy as np 
"""...................................................Hjorth parameters.................................................."""
def hjorth(a):
    first_deriv = np.diff(a)
    second_deriv = np.diff(a,2)

    var_zero = np.mean(a ** 2)
    var_d1 = np.mean(first_deriv ** 2)
    var_d2 = np.mean(second_deriv ** 2)

    activity = var_zero
    morbidity = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1) / morbidity

    return activity, morbidity, complexity
    import pandas as pd

"""...............................................Spectral_Entropy................................................."""

def spectral_entropy(x, sf, method='fft', nperseg=None, normalize=False):
    x = np.array(x)
    # Compute and normalize power spectrum
    if method == 'fft':
        _, psd = periodogram(x, sf)
    elif method == 'welch':
        _, psd = welch(x, sf, nperseg=nperseg)
    # add cond
    if psd.sum() == 0 :
        return np.nan
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        if psd_norm.size == 0:
            return np.nan
        se /= np.log2(psd_norm.size)
    return se
"""................................................Petrosian Fractal Diemension............................................"""
def petrosian_fd(x):
    n = len(x)
    # Number of sign changes in the first derivative of the signal
    diff = np.ediff1d(x)
    N_delta = (diff[1:-1] * diff[0:-2] < 0).sum()
    return np.log10(n) / (np.log10(n) + np.log10(n / (n + 0.4 * N_delta)))

"""..........................................  Absolute energy  .............................................."""
def abs_energy(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)

"""..........................................  Inter_Q_Range................................................"""
from scipy import stats
def inter_q_range(x):
  return stats.iqr(x)
"""..........................................median_absolute_deviation............................................."""
def median_absolute_deviation(data, axis=None, func=None, ignore_nan=False):
    if func is None:
        # Check if the array has a mask and if so use np.ma.median
        # See https://github.com/numpy/numpy/issues/7330 why using np.ma.median
        # for normal arrays should not be done (summary: np.ma.median always
        # returns an masked array even if the result should be scalar). (#4658)
        if isinstance(data, np.ma.MaskedArray):
            is_masked = True
            func = np.ma.median
            if ignore_nan:
                data = np.ma.masked_where(np.isnan(data), data, copy=False)
        elif ignore_nan:
            is_masked = False
            func = np.nanmedian
        else:
            is_masked = False
            func = np.median
    else:
        is_masked = None

    data = np.asanyarray(data)
    # np.nanmedian has `keepdims`, which is a good option if we're not allowing
    # user-passed functions here
    data_median = func(data, axis=axis)

    # broadcast the median array before subtraction
    if axis is not None:
        if isiterable(axis):
            for ax in sorted(list(axis)):
                data_median = np.expand_dims(data_median, axis=ax)
        else:
            data_median = np.expand_dims(data_median, axis=axis)

    result = func(np.abs(data - data_median), axis=axis, overwrite_input=True)

    if axis is None and np.ma.isMaskedArray(result):
        # return scalar version
        result = result.item()
    elif np.ma.isMaskedArray(result) and not is_masked:
        # if the input array was not a masked array, we don't want to return a
        # masked array
        result = result.filled(fill_value=np.nan)

    return result
"""..................................................... Shanon entopy..............................................................."""
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy = scipy.stats.entropy(probabilities)
    return entropy
"""..................................................... Standard statistics ......................................................."""
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.mean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
"""..................................................... Crossings........... .......................................................""" 
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

We now define a function called *get_features(list_values)* that returns for each signal a list of the computed features i.e the associated line in the design matrix.


In [ ]:
import h5py
import pywt
def get_features(list_values):
    a,b,c = hjorth(list_values)
    d = spectral_entropy(list_values, sf = 240)
    e = petrosian_fd(list_values)
    f = abs_energy(list_values)
    g = inter_q_range(list_values)
    h = median_absolute_deviation(list_values)
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics + [a,b,c,d,e,f,g,h]

The very next step is to construct the design matrix for both the training and test data. We will restrict ourself for only one eeg channel.

In [ ]:
with h5py.File("/content/drive/My Drive/X_train.h5", 'r') as f: 
  eeg_1 = f['eeg_1']
  n,p   = eeg_1.shape 
  list_fesature = []
  for k in range(n) :
    features = []
    features += get_features(eeg_1[k])
    list_features.append(features)
  X_train = np.array(X_train)
  np.savetxt('X_train.txt', X_train)

In [ ]:
with h5py.File("/content/drive/My Drive/X_test.h5", 'r') as f: 
  eeg_1 = f['eeg_1']
  n,p   = eeg_1.shape 
  list_fesature = []
  for k in range(n) :
    features = []
    features += get_features(eeg_1[k])
    list_features.append(features)
  X_test = np.array(X_test)
  np.savetxt('X_train.txt', X_test)

We download the labels of the training data and we transform it to a pure of classes.

In [ ]:
import pandas as pd
y = pd.read_csv("y_train.csv", sep= ',')
Y_train= []
for i in range(24684) :
  Y_train = Y_train + [y['sleep_stage'][i]]


Fro the training data we remove some lines for which the we don't have labels. As for None or inf values, we replace them by zero in both the training and the test data. 

In [ ]:
X_train = np.delete(X_train, obj= [i for i in range(24684, 24688)], axis =0)
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# 4. Algorithms 
Among the tested alhorithms, we restrict our discussion to the two most promising algorithms : Gradient Boosting classifier and Decision Tree with a Bagging classifier.

We run both of the algorithms without any tuninig of parameters. Most of the parameters keep their default values. The objective here is purely exploratory. 

## 4.1 Gradient Boosting classifier 
This algorithm consists in filtering sequentially observations to focus each time on developping a new learner that could deal with the difficult observations. The wealk learners take the form of the decision trees wich are constructed by making the best split accprding eith a purity score or a loss minimization. At each time a tree is added, a gradient decent procedure is used to min the loss function. The following code runs the algorithm on our data.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(  n_estimators= 100 )
clf.fit(X_train, Y_train)
Y_predict = clf.predict(X_test) 
Prediction = {'index' : [i + 24688 for i in range(len(Y_predict))], 'sleep_stage': Y_predict  }
Prediction = pd.DataFrame(Prediction , columns= [ 'index', 'sleep_stage'])
Prediction.to_csv (r'Predict_GradBoostMergeplus7ch08.csv', index = False ,header=True )

##4.2 Decision Tree with a Bagging classifier.
The bagging is an application of the bootstrap procedure to high-variance alhorithms. the training data is resampled into independent samples using for example an uniform sampling distribution. Thus, each model in the ensemble votes with equal weight. Each model in the ensemble is trained using a randomly drawn subset of the training set. The models are fitted using the created samples and are combined using voting (classification problem ). The decision tree algorithm can be enhanced when combined with a bagging classifier. The code below is an implementation of the latter procedure.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion = "entropy")
bag_model = BaggingClassifier(base_estimator=dtc, n_estimators=100, bootstrap=True)
bag_model = bag_model.fit(X_train, Y_train)
Y_predict = bag_model.predict(X_test)
Prediction = {'index' : [i + 24688 for i in range(len(Y_predict))], 'sleep_stage': Y_predict  }
Prediction = pd.DataFrame( Prediction, columns = [ 'index', 'sleep_stage']) 
Prediction.to_csv (r'Predict_BaggMergeplusentropy3ch.csv', index = False ,header=True )

## 4.3 Results : Single eeg channel.

The resulted $F_{1}$ scores are for each algorithm : 
*   Gradient Boosting classifier : 0.51
*   Decision Tree with a Bagging classifier : 0.58.

For the given design matrix, the Decision Tree with a Bagging classifier appears to be better. Our next move is try to redisign our matrix in a way that improves the scores without any hyperparameters tuning. In the following, we add some features related to the frequency domain of the signals.


# 5. Features from the frequency domain.




## 5.1 Signal transforms and their Python implementation
Features in the frequency domain are going to be essentially the highest peaks given by the signal. In this way, each new feature corresponds to a peak of the peak of the signal. The transformation of the signal to the frequency domain is ensured by three techniques :


*   Fast Fouriers Transform (FFT)
*   Power spectral density (PSD)
*   Autocorrelation function (Autocorr)

To determine the peaks of a given signal, we will use a function *detect_peaks* suggested by *Marcos Duarte* which one can find on github. This function returns the peaks on amplitude or other criterions. For our usage, We only detect peaks based on their amplitude. Our obejctive is detect only the highest five peaks for each transformation. *detect_peaks* will provide us each time with all the peaks greater than minimium hight set to be the 10% of the maximum hight. This condition is onlu an empirical way to ensure the existence of five peaks.

In [ ]:
"""Detect peaks in data based on their amplitude and other features."""

from __future__ import division, print_function
import numpy as np

__author__ = "Marcos Duarte, https://github.com/demotu/BMC"
__version__ = "1.0.6"
__license__ = "MIT"


def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None, title=True):

    """Detect peaks in data based on their amplitude and other features.

    Parameters
    ----------
    x : 1D array_like
        data.
    mph : {None, number}, optional (default = None)
        detect peaks that are greater than minimum peak height (if parameter
        `valley` is False) or peaks that are smaller than maximum peak height
         (if parameter `valley` is True).
    mpd : positive integer, optional (default = 1)
        detect peaks that are at least separated by minimum peak distance (in
        number of data).
    threshold : positive number, optional (default = 0)
        detect peaks (valleys) that are greater (smaller) than `threshold`
        in relation to their immediate neighbors.
    edge : {None, 'rising', 'falling', 'both'}, optional (default = 'rising')
        for a flat peak, keep only the rising edge ('rising'), only the
        falling edge ('falling'), both edges ('both'), or don't detect a
        flat peak (None).
    kpsh : bool, optional (default = False)
        keep peaks with same height even if they are closer than `mpd`.
    valley : bool, optional (default = False)
        if True (1), detect valleys (local minima) instead of peaks.
    show : bool, optional (default = False)
        if True (1), plot data in matplotlib figure.
    ax : a matplotlib.axes.Axes instance, optional (default = None).
    title : bool or string, optional (default = True)
        if True, show standard title. If False or empty string, doesn't show
        any title. If string, shows string as title.

    Returns
    -------
    ind : 1D array_like
        indeces of the peaks in `x`.

    Notes
    -----
    The detection of valleys instead of peaks is performed internally by simply
    negating the data: `ind_valleys = detect_peaks(-x)`

    The function can handle NaN's

    See this IPython Notebook [1]_.

    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
    """

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
        if mph is not None:
            mph = -mph
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                       & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
            if mph is not None:
                mph = -mph
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind, title)

    return ind


The following two functions ensure returning for a given signal transformation the x-values and y-values of the five peaks. We thus have extrcat ten features from each transformation.

In [ ]:
def get_first_n_peaks(x,y,no_peaks=5):
    x_, y_ = list(x), list(y)
    if len(x_) >= no_peaks:
        return x_[:no_peaks], y_[:no_peaks]
    else:
        missing_no_peaks = no_peaks-len(x_)
        return x_ + [0]*missing_no_peaks, y_ + [0]*missing_no_peaks
    
def get_features(x_values, y_values, mph):
    indices_peaks = detect_peaks(y_values, mph=mph)
    peaks_x, peaks_y = get_first_n_peaks(x_values[indices_peaks], y_values[indices_peaks])
    return peaks_x + peaks_y

We implement below the three considered transformations.

In [ ]:
def get_psd_values(y_values, T, N, f_s):
    f_values, psd_values = welch(y_values, fs=f_s) # return freq values and ampl
    return f_values, psd_values

def get_fft_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:]
 
def get_autocorr_values(y_values, T, N, f_s):
    autocorr_values = autocorr(y_values)
    x_values = np.array([T * jj for jj in range(0, N)])
    return x_values, autocorr_values

We now extract the additional features for the training and the test data. Based on similarity, it suffices to present the code applied to the training data.

In [ ]:
N = 1500
f_s = 240 # sampling freq
T = 1/f_s
with h5py.File("/content/drive/My Drive/X_train.h5", 'r') as f: 
  eeg_i = f['eeg_' + str(1)]
  n,p   = eeg_i.shape 
  Data = []
  for k in range(n):
    # Apply PSD
    x_values, y_values = get_psd_values(eeg_i[k], T, N, f_s)
    max_peak_height = 0.1 * np.nanmax(y_values)
    PSD = get_features(x_values, y_values, mph = max_peak_height)
    # Apply FFT
    x_values, y_values = get_fft_values(eeg_i[k], T, N, f_s)
    max_peak_height = 0.1 * np.nanmax(y_values)
    FFT = get_features(x_values, y_values, mph = max_peak_height)
    # Autocorr
    x_values, y_values = get_autocorr_values(eeg_i[k], T, N, f_s)
    max_peak_height = 0.1 * np.nanmax(y_values)
    Autocorr = get_features(x_values, y_values, mph = max_peak_height)
    Features = FFT + PSD + Autocorr
    Data = Data + [Features]

The additiona features are simply added as new columns in our initial design matrix. We apply the same two algorithms to our new data.

## 5.2 Results : One single eeg channel
The resulted $F_{1}$ scores are for each algorithm : 
*   Gradient Boosting classifier : 0.53
*   Decision Tree with a Bagging classifier : 0.61.
We observe an improvement of the score compared to the initial ones but not very sensitive. The Decision Tree with a Bagging classifier is still better at this step.
The fact that the added features did not improve the scores in a spectacualr way is mailnu due to the fact the esential tool of transformation -The Fourier transform- requires the frequency domain to be stationary. Unfortunately, the eeg signals appear to be dyanmic in the litterature. We thus need a tool that could provide simulataneously with the interesting frequencies and the time resolution at which they occur. Our next step is to perform wavelet transforms to our eeg signals.

# 6. Wavelet transform and features extraction.
The Fourier Transform has a high resolution in the frequency-domain but zero resolution in the time-domain. This means that it can tell us exactly which frequencies are present in a signal, but not at which location in time these frequencies have occurred. 
A better approach for analyzing signals with a dynamical frequency spectrum is the Wavelet Transform which has a high resolution in both the frequency- and the time-domain. It does not only tell us which frequencies are present in a signal, but also at which time these frequencies have occurred. This is accomplished by working with different scales. First, we look at the signal with a large scale/window and analyze ‘large’ features and then we look at the signal with smaller scales in order to analyze smaller features.
Wavelets are mathematical functions that resemble the shape of wave oscillations, with the constraint for waves to start at 0 and then oscillate to 0 in the end. Mathematically, wavelets are described using two types of functions: the wavelet function (the mother function denoted with ψ(t)) and the scaling function (the father function denoted with φ(t)). These functions have to satisfy some conditions of integribality which we wills skip here.

The application of the wavelet transform (in our case the discrete ) on the signal results in frequency sub-bands that capture some characteristics of the signal. For each sub-bands, we will extract the same features defined fro the initial design matrix.The discrete wavelet transforms used are : db1, db2, db3,db4 and haar.


In the following, we define a function that generate features for each frequency sub-bands given a wavelet tranform. *eeg_data* reprsents here the list of all the signals. This function is applied to both training signal and test signals.

In [ ]:
def get_eeg_features(eeg_data,waveletname):
    list_features = []
    for i, signal in enumerate(eeg_data):
        # print(i)
        list_coeff = pywt.wavedec(signal, waveletname)
        features = []
        for coeff in list_coeff:
            features += get_features(coeff)
        list_features.append(features)
    return list_features

An example of features extraction is given below for one eeg channel. We use the wavelet db3

In [ ]:
with h5py.File("/content/drive/My Drive/X_train.h5", 'r') as f: 
  eeg_1 = f['eeg_1']
  n,p   = eeg_1.shape 
  X_train = get_eeg_features(eeg_data = eeg_1 ,waveletname = 'db3')
  X_train = np.array(X_train)
  np.savetxt('X_train_eeg1_db3.txt', X_train)

## 6.2 Results

### 6.2.1 : Application of a single wavelet transform to a single eeg channel
Once our training and test design matrices are constructed we run the same two algorithms. We apply seperately the wavelets db1 and db3.

The resulted $F_{1}$ scores are for each algorithm : 

**For db1 :**
*   Gradient Boosting classifier : 0.64
*   Decision Tree with a Bagging classifier : 0.62.

**For db3 :**
*   Gradient Boosting classifier : 0.66
*   Decision Tree with a Bagging classifier : 0.64.

Using the wavelet transform, the improvement of the scores is quite sensitive. the Gradient Boosting classifier starts take the lead but the the resulted depens on the wavelet transform.

### 6.2.2 Application of mutilple wavelet transforms to a single eeg channel.

In this step we suggest to apply the five considered wavelet transforms and cancatenate vertically the resulted columns in the design matrix. We will restrict ourself to one eeg channel.

The resulted $F_{1}$ scores are for each algorithm : 
*   Gradient Boosting classifier : 0.69
*   Decision Tree with a Bagging classifier : 0.66.

We observe an enhancement of the performance with the Gradient Boosting classifier as the leader.


### 6.2.3 Application of mutilple wavelet transforms to multiple eeg channels.
Our next move is to apply the same multiple wavelts transform for mutliple eeg channels. Actually the data provided contains 7 eeg channels. we will use them all.

The resulted $F_{1}$ scores are for each algorithm : 
*   Gradient Boosting classifier : 0.73
*   Decision Tree with a Bagging classifier : 0.71

# 7. Hyper-parameters tuning for the best algorithm : Gradient Boosting classifier.


## 7.1 The tuned parameters with Python implementation

We suggest to fine tune three parameters: 

*   n_estimators : Numbers of the modeled sequential trees
*   max_depth : maximul depth of the tree.
*   min_samples_split : Minimum number of samples in leaf

The search for the optimum paarameters is performed by grid search. We use *GridSearchCV* from sklearn library. We fix the learning rate at the default value 0.1 we look firstly at the optimal number of estimators. The other paameters are either set intuitively or following recommanded values like 0.1 for the learning_rate and 0.8 for subsample. 25% of features as for max_features needed to make a split.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.model_selection import GridSearchCV  # Performing grid search
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV( estimator = GradientBoostingClassifier( learning_rate=0.1, min_samples_split=500, min_samples_leaf= 50,
max_depth=8, max_features='sqrt', subsample = 0.8 , random_state = 10), param_grid = param_test1, scoring= 'accuracy' ,iid=False, cv = 10)
gsearch1.fit(X_train ,Y_train)

The optimal number of estimators is n_estimators = 80. We use this optimal parameter value to tune tree specific parameters : max_depth and min_samples_split.

In [ ]:
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
gsearch2 = GridSearchCV (estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators = 80 , max_features='sqrt'
, subsample = 0.8, random_state=10), 
param_grid = param_test2, scoring = 'accuracy',iid=False, cv = 10)
gsearch2.fit(X_train ,Y_train)

The optimal values are  : max_depth : 15, min_samples_split : 800.


## 7.2 Results of the tuned algorithm.
We manage finally, by tuning the Gradient Boosting Classifier, to achieve the $F_1$ score of 0.764.